In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datetime import timedelta
# chartevents.csv는 27.79GB 이므로 한번에 RAM에 올릴 수 없음. 따라서 청크 단위로 나누어 작업 진행
# 중환자실 입실 시간을 포함한 icustays 데이터 불러오기
df_icustays = pd.read_csv('/content/drive/MyDrive/DA_dataset/icustays.csv', usecols=['stay_id', 'intime'], parse_dates=['intime'])

# 결과를 저장할 리스트
filtered_data = []

# 청크 단위로 chartevents 데이터 읽기
chunk_size = 1_000_000  # 청크 크기 설정
for chunk in pd.read_csv('/content/drive/MyDrive/DA_dataset/chartevents.csv', chunksize=chunk_size, parse_dates=['charttime'],dtype={'value': object, 'valueuom': object},low_memory=False):
    # icustays 데이터와 결합하여 각 행에 중환자실 입실 시간 추가
    merged_chunk = chunk.merge(df_icustays, on='stay_id', how='inner')

    # 중환자실 입실 후 6시간 이내의 이벤트 필터링
    merged_chunk['endtime'] = merged_chunk['intime'] + timedelta(hours=6)  # 6시간 후 시간 계산
    filtered_chunk = merged_chunk[(merged_chunk['charttime'] >= merged_chunk['intime']) &
                                  (merged_chunk['charttime'] <= merged_chunk['endtime'])]

    # 필터링된 데이터 저장
    filtered_data.append(filtered_chunk)

# 모든 청크 결합
df_filtered = pd.concat(filtered_data, ignore_index=True)

#csv로 저장
df_filtered.to_csv('/content/drive/MyDrive/DA_dataset/18hour_chartevents.csv', index=False)

<ipython-input-3-51f0bbadc661>:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in pd.read_csv('/content/drive/MyDrive/DA_dataset/chartevents.csv', chunksize=chunk_size, parse_dates=['charttime'],dtype={'value': object, 'valueuom': object},low_memory=False):


In [ ]:
# 6시간 이내 데이터만 필터링할 함수 정의
def filter_6hours_no_chunks(file_path, time_column):
    # 파일 전체를 한 번에 읽어옴
    df = pd.read_csv(file_path, parse_dates=[time_column])

    # icustays 데이터와 병합하여 중환자실 입실 시간 추가
    merged_df = df.merge(df_icustays, on='stay_id', how='inner')

    # 중환자실 입실 후 6시간 이내의 데이터 필터링
    merged_df['endtime'] = merged_df['intime'] + timedelta(hours=6)
    filtered_df = merged_df[(merged_df[time_column] >= merged_df['intime']) &
                            (merged_df[time_column] <= merged_df['endtime'])]
    return filtered_df

# 각 테이블에서 6시간 이내 데이터만 추출
inputevents_6hours = filter_6hours_no_chunks('/content/drive/MyDrive/DA_dataset/inputevents.csv', 'starttime')

# 사용 안함
#outputevents_6hours = filter_6hours_no_chunks('/content/drive/MyDrive/DA_dataset/outputevents.csv', 'charttime')
#procedureevents_6hours = filter_6hours_no_chunks('/content/drive/MyDrive/DA_dataset/procedureevents.csv', 'starttime')

# 필요 시, 각 테이블을 CSV로 저장
inputevents_6hours.to_csv('/content/drive/MyDrive/DA_dataset/6hour_inputevents.csv', index=False)
# outputevents_6hours.to_csv('/content/drive/MyDrive/DA_dataset/6hour_outputevents.csv', index=False)
# procedureevents_6hours.to_csv('/content/drive/MyDrive/DA_dataset/6hour_procedureevents.csv', index=False)